# Company Prices And Market Caps

### Description 

The following code leverages the yfinance and pandas_datareader package and data scarped from https://ycharts.com/. to gather average historical stock price and market cap data for 3,225 companies.

The stock price data are historical prices from 2000/12 - 2019/12. 

To capture a specific year's stock price return, the historical stock price data for prices in December are averaged and multiplied by the companies' average diluted shares outstanding for the given year (market cap). <br><br>
With the companies' market cap data, compounded return from year 2000 - 2019 can be calculated<br><br>

The market cap data will be concatenated with the the balance sheet, income, and cash flow statement data queried through the EDGAR API.

### Data Gathering and Preprocessing

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import yahoo_fin.stock_info as si
from pandas_datareader import data
from ipynb.fs.full.Preprocessing_Functions import *

In [80]:
# Import a list of stock tickers
wil_df_fin = pd.read_csv('Data/Wilshire_5000_All_Holdings.csv')

# Check data Summary
print('\n')
wil_df_fin.info(null_counts = True)
wil_df_fin.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3225 entries, 0 to 3224
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Ticker  3225 non-null   object
 1   Name    3221 non-null   object
 2   Sector  3217 non-null   object
dtypes: object(3)
memory usage: 75.7+ KB


,Ticker,Name,Sector
0,A,Agilent Technologies Inc.,Healthcare
1,AA,Alcoa Corp,Basic Materials
2,AAL,American Airlines Group Inc,Industrials
3,AAME,Atlantic American Corp.,Financial Services
4,AAN,Aarons Company Inc (The),Industrials


A data frame of stock tickers and their company names is imported. The list of stock tickers is specific and reduced from the original list of stocks to accommodate the data limitations and de-listings of stocks. <br>

In [116]:
# List of company ticker symbols
ticker_list = wil_df_fin['Ticker'].unique()

# List to store loaded data frames
# for loop to generate prices  for every company found in the wil_df data frame
# List to hold all created data frames
price_list = []
for ticker in log_progress(ticker_list):
    try:
        # Search for prices by ticker
        df = yf.Ticker(ticker)
        # Selecet start date
        df = df.history(start = "2000-12-01")
        # Reset index
        df = df.reset_index()
        # Creat a new column for stock ticker
        df['company_ticker'] = df['company_ticker'] = ticker
        # import package to retrive market cap data
        from pandas_datareader import data
        # retrive market cap data per ticker
        market_cap = data.get_quote_yahoo(ticker)['marketCap']
        # Convert the retrived market cap data into a data frame
        market_cap = pd.DataFrame(market_cap).reset_index()
        # Rename columns
        market_cap.columns = ['company_ticker','market_cap']
        # Merge stock price and market cap data frames
        df = pd.merge(df, market_cap, on = 'company_ticker')
        # Drop columns
        df.columns = ['date','open','high','low','close','volume','dividends','stock_splits','company_ticker',
                      'market_cap']
        # Append df to wil_list
        price_list.append(df)
    except (RuntimeError, KeyError, IndexError, ValueError):
        continue 
        
# Concatonate all data frames in the wil_list           
price = pd.concat(price_list)

# Check data Summary
print('\n')
price.info()
price.head()

- AIII: No data found for this date range, symbol may be delisted
- AIMT: No data found, symbol may be delisted
- AKCA: No data found, symbol may be delisted
- BSTC: No data found, symbol may be delisted
- CBSA: No data found for this date range, symbol may be delisted
- DNKN: No data found, symbol may be delisted
- FRAN: No data found, symbol may be delisted
- FSCT: No data found, symbol may be delisted
- GLIBA: No data found, symbol may be delisted
- GPOR: No data found, symbol may be delisted
- HDS: No data found, symbol may be delisted
- HNR: No data found for this date range, symbol may be delisted
- IRET: No data found, symbol may be delisted
- PIH: No data found, symbol may be delisted
- PRCP: No data found, symbol may be delisted
- PTI: No data found, symbol may be delisted
- ROKA: No data found for this date range, symbol may be delisted
- TAT: No data found, symbol may be delisted
- TECD: No data found, symbol may be delisted
- TLF: No data found, symbol may be delisted
- TTS

,date,open,high,low,close,volume,dividends,stock_splits,company_ticker,market_cap
0,2000-12-01,32.740625,33.822957,32.624660,32.779278,3628649.0,0.0,0.0,A,37287997440
1,2000-12-04,32.199465,32.817943,31.233094,31.542334,2895118.0,0.0,0.0,A,37287997440
2,2000-12-05,32.624662,34.441438,32.508697,34.054890,5472331.0,0.0,0.0,A,37287997440
3,2000-12-06,34.170871,35.214550,32.160820,32.160820,4366933.0,0.0,0.0,A,37287997440
4,2000-12-07,32.470059,33.938940,31.774271,33.397774,3059942.0,0.0,0.0,A,37287997440


A for loop is used to gather all the historical stock price and market cap data using the imported list of stock tickers. <br>
The years of historical stock price data will be reduced to range from 2000/12/01 - 2019/12/01.<br>

In [117]:
# Export price data frame to project data directory
price.to_csv('Data/2000_2019_Wilshire_5000_Prices_Market_Caps.csv')

I export the wil_df data frame to the project directory because of the large size of the file.<br>

In [3]:
# Import data frame
price = pd.read_csv('Data/2000_2019_Wilshire_5000_Prices_Market_Caps.csv')
# Drop Unnamed: 0
price = price.drop(['Unnamed: 0'], axis = 1).reset_index(drop = True)

# Check data Summary
print('\n')
price.info(null_counts = True)
price.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11853127 entries, 0 to 11853126
Data columns (total 10 columns):
 #   Column          Non-Null Count     Dtype  
---  ------          --------------     -----  
 0   date            11853127 non-null  object 
 1   open            11853099 non-null  float64
 2   high            11853099 non-null  float64
 3   low             11853099 non-null  float64
 4   close           11853099 non-null  float64
 5   volume          11853099 non-null  float64
 6   dividends       11853127 non-null  float64
 7   stock_splits    11853127 non-null  float64
 8   company_ticker  11853127 non-null  object 
 9   market_cap      11853127 non-null  int64  
dtypes: float64(7), int64(1), object(2)
memory usage: 904.3+ MB


,date,open,high,low,close,volume,dividends,stock_splits,company_ticker,market_cap
0,2000-12-01,32.740625,33.822957,32.624660,32.779278,3628649.0,0.0,0.0,A,37287997440
1,2000-12-04,32.199465,32.817943,31.233094,31.542334,2895118.0,0.0,0.0,A,37287997440
2,2000-12-05,32.624662,34.441438,32.508697,34.054890,5472331.0,0.0,0.0,A,37287997440
3,2000-12-06,34.170871,35.214550,32.160820,32.160820,4366933.0,0.0,0.0,A,37287997440
4,2000-12-07,32.470059,33.938940,31.774271,33.397774,3059942.0,0.0,0.0,A,37287997440


The wil_df data frame is imported back into the analysis. <br>

In [4]:
# Ignore unneeded warnings
import warnings
warnings.filterwarnings("ignore")

# Return all NaN values
null_value = price[price.isnull().any(axis=1)]
# Create a column for the month
null_value['month'] = pd.DatetimeIndex(null_value['date']).month
# Filter for prices only in December 
null_value = null_value.loc[null_value['month'] == 12]
null_value

,date,open,high,low,close,volume,dividends,stock_splits,company_ticker,market_cap,month


Extracted and observed the NaN values for dates only in December because these are the historical stock prices that will be used.<br>

In [5]:
# Creat a new column for the year
price['year'] = pd.DatetimeIndex(price['date']).year
# Creat a column for the month
price['month'] = pd.DatetimeIndex(price['date']).month
# Filter for prices only in December and within year range 2010 - 2019
price_00_19 = price.loc[(price['month'] == 12) & (price['year'] != 2020)].reset_index()
# Drop NaN
price_00_19 = price_00_19.dropna()

# Check data Summary
print('\n')
price_00_19.info(null_counts = True)
price_00_19.head()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 956145 entries, 0 to 956144
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   index           956145 non-null  int64  
 1   date            956145 non-null  object 
 2   open            956145 non-null  float64
 3   high            956145 non-null  float64
 4   low             956145 non-null  float64
 5   close           956145 non-null  float64
 6   volume          956145 non-null  float64
 7   dividends       956145 non-null  float64
 8   stock_splits    956145 non-null  float64
 9   company_ticker  956145 non-null  object 
 10  market_cap      956145 non-null  int64  
 11  year            956145 non-null  int64  
 12  month           956145 non-null  int64  
dtypes: float64(7), int64(4), object(2)
memory usage: 102.1+ MB


,index,date,open,high,low,close,volume,dividends,stock_splits,company_ticker,market_cap,year,month
0,0,2000-12-01,32.740625,33.822957,32.624660,32.779278,3628649.0,0.0,0.0,A,37287997440,2000,12
1,1,2000-12-04,32.199465,32.817943,31.233094,31.542334,2895118.0,0.0,0.0,A,37287997440,2000,12
2,2,2000-12-05,32.624662,34.441438,32.508697,34.054890,5472331.0,0.0,0.0,A,37287997440,2000,12
3,3,2000-12-06,34.170871,35.214550,32.160820,32.160820,4366933.0,0.0,0.0,A,37287997440,2000,12
4,4,2000-12-07,32.470059,33.938940,31.774271,33.397774,3059942.0,0.0,0.0,A,37287997440,2000,12


All the historical stock prices for the month of December are extracted.<br>

In [6]:
# Drop unneeded coulmns
price_00_19 = price_00_19.drop(['index','date','open','high','low','volume','dividends','stock_splits',
                                'month'], axis = 1, errors = 'ignore')

# Check data Summary
print('\n')
price_00_19.info(null_counts = True)
price_00_19.head()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 956145 entries, 0 to 956144
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   close           956145 non-null  float64
 1   company_ticker  956145 non-null  object 
 2   market_cap      956145 non-null  int64  
 3   year            956145 non-null  int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 36.5+ MB


,close,company_ticker,market_cap,year
0,32.779278,A,37287997440,2000
1,31.542334,A,37287997440,2000
2,34.054890,A,37287997440,2000
3,32.160820,A,37287997440,2000
4,33.397774,A,37287997440,2000


All unneeded columns are dropped.<br>

In [7]:
# Group by Year and Company Ticker for and calculate the mean prices
price_00_19 = price_00_19.groupby(['year', 'company_ticker']).mean().reset_index()
# drop columns
price_00_19 = price_00_19.drop(['market_cap'], axis = 1)
# Change column names
price_00_19.columns = ['date', 'ticker', 'close']
# Reorder columns
price_00_19 = price_00_19[['ticker', 'date', 'close']]
# Create unique id
price_00_19['id'] = price_00_19['ticker'] + price_00_19['date'].astype(str)

# Check data Summary
print('\n')
price_00_19.info(null_counts = True)
price_00_19.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45737 entries, 0 to 45736
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ticker  45737 non-null  object 
 1   date    45737 non-null  int64  
 2   close   45737 non-null  float64
 3   id      45737 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 1.4+ MB


,ticker,date,close,id
0,A,2000,34.199850,A2000
1,AA,2000,57.926965,AA2000
2,AAME,2000,1.786134,AAME2000
3,AAON,2000,1.526483,AAON2000
4,AAPL,2000,0.230242,AAPL2000


The stock prices are averaged for the year per stock ticker; this will give us the average stock price in December for a given year and stock ticker. <br>

In [21]:
# Import data
shares_out = pd.read_csv('Data/Shares_Outstanding.csv').drop(['Unnamed: 0'], axis = 1).reset_index(drop = True)
# Create Unique id
shares_out['id'] = shares_out['ticker'] + shares_out['date']
# Empty list
shares_out_list = []
# Create a data frame to fit data to
for ticker in shares_out['ticker'].unique().tolist():
    for x in range (1,21):
        data = {'ticker': ticker,
                'date': 2020 - x}
        shares_out_list.append(data)

# Convert empty list to data frame
shares_out_list = pd.DataFrame(shares_out_list)
# Create unique id
shares_out_list['id'] = shares_out_list['ticker'] + shares_out_list['date'].astype(str)
# Merge both data frames
shares_out = shares_out_list.merge(shares_out, how = 'left', on = 'id')
# drop columns
shares_out = shares_out.drop(['ticker_y', 'date_y'], axis = 1)
# Rename columns
shares_out.columns = ['ticker', 'date', 'id', 'shares_outstanding']
# Remove duplicates and keep first instance
shares_out = shares_out.drop_duplicates(subset = ['id'], keep = "first")
# Fill na values with 0
shares_out = shares_out.fillna(0)
# Replace all 'Maximumm' values with 0
shares_out['shares_outstanding'] = shares_out['shares_outstanding'].replace('Maximum', 0)
# Remove the last character in column
shares_out['shares_outstanding_clean'] = shares_out['shares_outstanding'].astype(str).map(lambda x: str(x)[: -1])
# Replace blank values with 0
shares_out['shares_outstanding_clean'] = shares_out['shares_outstanding_clean'].replace('', 0)
# Convert column to data type float
shares_out['shares_outstanding_clean'] = shares_out['shares_outstanding_clean'].astype(float)
# Keep last characater and save to a new column
shares_out['unit'] = [x.strip()[-1] for x in shares_out['shares_outstanding'].astype(str)]
# Replace values
shares_out['unit'] = shares_out['unit'].replace('T', 1000000000000)
shares_out['unit'] = shares_out['unit'].replace('B', 1000000000)
shares_out['unit'] = shares_out['unit'].replace('M', 1000000)
# Convert column to data type float
shares_out['unit'] = shares_out['unit'].astype(float)
# Drop old column
shares_out = shares_out.drop(['shares_outstanding'], axis = 1, errors = 'ignore')
# Multiply value by unit in new column
shares_out['shares_outstanding'] = shares_out['shares_outstanding_clean'] * shares_out['unit']
# Drop old column
shares_out = shares_out.drop(['shares_outstanding_clean', 'unit'], axis = 1, errors = 'ignore')
# Create unique id
shares_out['id'] = shares_out['ticker'] + shares_out['date'].astype(str)

# Print data summary
print('\n')
shares_out.info()
shares_out.head()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 63300 entries, 0 to 64395
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ticker              63300 non-null  object 
 1   date                63300 non-null  int64  
 2   id                  63300 non-null  object 
 3   shares_outstanding  63300 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 2.4+ MB


,ticker,date,id,shares_outstanding
0,A,2019,A2019,318000000.0
1,A,2018,A2018,325000000.0
2,A,2017,A2017,326000000.0
3,A,2016,A2016,329000000.0
4,A,2015,A2015,335000000.0


Shares outstanding data is cleaned.

In [22]:
# Merge all data frames
data = shares_out.merge(price_00_19, how = 'left', on = 'id')
# Reorder columns
data = data[['ticker_x','date_x', 'id', 'close','shares_outstanding']]
data.columns = ['ticker','date', 'id', 'close','shares_outstanding']
# fill na with 0
data = data.fillna(0)

# Calculate market cap
data['market_cap'] = data['close'] * data['shares_outstanding']
data['statement'] = 'Other'
data = data.reset_index(drop = True)

# Unpivot data
data = pd.melt(data, id_vars = ['ticker', 'date', 'id', 'statement'])
data = data.drop(['id'], axis = 1)
data.columns = ['company_ticker','year','statement','field', 'value']

# Print data summary
print('\n')
data.info()
data.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189900 entries, 0 to 189899
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   company_ticker  189900 non-null  object 
 1   year            189900 non-null  int64  
 2   statement       189900 non-null  object 
 3   field           189900 non-null  object 
 4   value           189900 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 7.2+ MB


,company_ticker,year,statement,field,value
0,A,2019,Other,close,82.730826
1,A,2018,Other,close,67.296297
2,A,2017,Other,close,65.265496
3,A,2016,Other,close,43.950150
4,A,2015,Other,close,39.174646


All data sets are merged.<br>
Closing prices and average annual diluted shares outstanding are multiplied to get the approximate market-cap for the given year. <br>


In [23]:
# Import data
wil_df = pd.read_csv('Data/2000_2019_Wilshire_5000_Financials.csv')
# Remove unneeded column
wil_df = wil_df.drop(['Unnamed: 0'], axis = 1)
# Create a list of unique years
year_list = wil_df['year'].unique()[0:20]
# Filter for years in year list
wil_df = wil_df[wil_df['year'].isin(year_list)].reset_index(drop = True)

# Print data summary
print('\n')
wil_df.info(null_counts = True)
print('\n' + 'sum of duplicate records: ' + '\n' + str(wil_df.duplicated().sum()) + '\n')
wil_df.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3261053 entries, 0 to 3261052
Data columns (total 6 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   year            3261053 non-null  int64  
 1   company_ticker  3261053 non-null  object 
 2   company_name    3261053 non-null  object 
 3   statement       3261053 non-null  object 
 4   field           3261053 non-null  object 
 5   value           3261053 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 149.3+ MB

sum of duplicate records: 
0



,year,company_ticker,company_name,statement,field,value
0,2019,A,Agilent Technologies Inc.,BalanceSheet,accountspayableandaccruedexpenses,1.006000e+09
1,2019,A,Agilent Technologies Inc.,BalanceSheet,additionalpaidincapital,5.311000e+09
2,2019,A,Agilent Technologies Inc.,BalanceSheet,cashandcashequivalents,1.441000e+09
3,2019,A,Agilent Technologies Inc.,BalanceSheet,cashcashequivalentsandshortterminvestments,1.441000e+09
4,2019,A,Agilent Technologies Inc.,BalanceSheet,commonstock,3.000000e+06


In [24]:
company_names = wil_df.copy()
company_name = company_names[['company_ticker','company_name']]
company_name = company_name.drop_duplicates().reset_index(drop = True)
data = data.merge(company_name, on = 'company_ticker')
data = data[['year','company_ticker','company_name','statement','field','value']]
data = pd.concat([wil_df, data])

print('\n')
data.info(null_counts = True)
data.head()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 3448193 entries, 0 to 187139
Data columns (total 6 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   year            3448193 non-null  int64  
 1   company_ticker  3448193 non-null  object 
 2   company_name    3448193 non-null  object 
 3   statement       3448193 non-null  object 
 4   field           3448193 non-null  object 
 5   value           3448193 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 184.2+ MB


,year,company_ticker,company_name,statement,field,value
0,2019,A,Agilent Technologies Inc.,BalanceSheet,accountspayableandaccruedexpenses,1.006000e+09
1,2019,A,Agilent Technologies Inc.,BalanceSheet,additionalpaidincapital,5.311000e+09
2,2019,A,Agilent Technologies Inc.,BalanceSheet,cashandcashequivalents,1.441000e+09
3,2019,A,Agilent Technologies Inc.,BalanceSheet,cashcashequivalentsandshortterminvestments,1.441000e+09
4,2019,A,Agilent Technologies Inc.,BalanceSheet,commonstock,3.000000e+06


Both market cap and financial statement data are concatenated into one data frame.

In [25]:
# IMport data frame
fields = pd.read_csv('Data/Field_Data.csv')

# Print data summary
print('\n')
fields.info()
fields.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109 entries, 0 to 108
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   field          109 non-null    object
 1   cleaned_field  109 non-null    object
dtypes: object(2)
memory usage: 1.8+ KB


,field,cleaned_field
0,accountspayableandaccruedexpenses,Accounts Payable And Accrued Expenses
1,additionalpaidincapital,Additional Paid In Capital
2,cashandcashequivalents,Cash And Cash Equivalents
3,cashcashequivalentsandshortterminvestments,Cash Cashequivalents And Short Term Investments
4,commonstock,Common Stock


Import cleaned fields column.

In [26]:
# Merge data sets
data = data.merge(fields, on = 'field')
# Drop unneeded columns
data = data.drop(['field'], axis = 1, errors = 'ignore')
# Reorder columns
data = data[['year','company_ticker','company_name','statement','cleaned_field','value']]
# Rename columns
data.columns = ['year','company_ticker','company_name','statement','field','value']

# Print data summary
print('\n')
data.info(null_counts = True)
data.head()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 3448193 entries, 0 to 3448192
Data columns (total 6 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   year            3448193 non-null  int64  
 1   company_ticker  3448193 non-null  object 
 2   company_name    3448193 non-null  object 
 3   statement       3448193 non-null  object 
 4   field           3448193 non-null  object 
 5   value           3448193 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 184.2+ MB


,year,company_ticker,company_name,statement,field,value
0,2019,A,Agilent Technologies Inc.,BalanceSheet,Accounts Payable And Accrued Expenses,1.006000e+09
1,2018,A,Agilent Technologies Inc.,BalanceSheet,Accounts Payable And Accrued Expenses,1.128000e+09
2,2017,A,Agilent Technologies Inc.,BalanceSheet,Accounts Payable And Accrued Expenses,8.470000e+08
3,2016,A,Agilent Technologies Inc.,BalanceSheet,Accounts Payable And Accrued Expenses,7.620000e+08
4,2015,A,Agilent Technologies Inc.,BalanceSheet,Accounts Payable And Accrued Expenses,6.760000e+08


Merge cleaned fields to data frame.

In [29]:
# BalanceSheet
balance_sheet = data.loc[data['statement'] == 'BalanceSheet']
# IncomeStatement
income_statement = data.loc[data['statement'] == 'IncomeStatement']
# CashFlowStatement
cash_flow_statement = data.loc[data['statement'] == 'CashFlowStatement']
# Other
other = data.loc[data['statement'] == 'Other']

Split the date frame by statements for faster processing.

In [30]:
# Export data to project data directory
data.to_csv('Data/2000_2019_Wilshire_5000_Historical_Company_Data.csv')
balance_sheet.to_csv('Data/Balance_Sheet_Data.csv')
income_statement.to_csv('Data/Income_Statement_Data.csv')
cash_flow_statement.to_csv('Data/Cash_Flow_Statement_Data.csv')
other.to_csv('Data/Other_Data.csv')

The finished data frame is exported to the project directory for later use.  <br>